<a href="https://colab.research.google.com/github/1Asfandyar/Smart-Attendance-System/blob/AI-model/ML_model/ML_Model_Function.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Library installation**

In [1]:
!pip install face_recognition
!pip install -v --install-option="--no" --install-option="DLIB_USE_CUDA" dlib
!pip install mtcnn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 100.1 MB 22 kB/s 
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566186 sha256=117a54fe42d5e3dbccd21a054847700342a6072e120f304ffbbb6f4d4a9b9658
  Stored in directory: /root/.cache/pip/wheels/d6/81/3c/884bcd5e1c120ff548d57c2ecc9ebf3281c9a6f7c0e7e7947a
Successfully built face-recognition-models
/usr/local/lib/python3.7/dist-packages/pip/_internal/commands/install.py:232: UserWarning: Disabling all use of wheels due to the use of --build-option / --global-option / --install-option.
  cmdoptions.check_install_build_global(options)
Using pip 21.1.3 from /usr/local/lib/python3.7/dist-packages/pip (python 3.7)
Value for scheme.platlib does not match. Please report this to <https://github.com/pypa/pip/issues/9617>
distutils: /usr/local/lib/python3.7/dist-packages
sysconfig: /usr/lib/python3.7

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import face_recognition
import matplotlib.pyplot as plt
import cv2
import numpy as np
import os
from datetime import datetime
from PIL import Image, ImageDraw, ImageFont
import pickle
from matplotlib import pyplot
from matplotlib.patches import Rectangle
from matplotlib.patches import Circle
from mtcnn.mtcnn import MTCNN

Function that create encoding of all faces from specific folder.
Then save that in pkl file in the drive.

In [4]:
def savePickleEncodingsToFile(file_name:str, py_obj):
  """
  This function saves a Python Object of any type in Pickle encoded form to a file.
  INPUTS:
    file_name:
      name of the file to which encodings have to store.
    py_obj:
      Any Python object which is to be stored in pickle form to the file.
  """
  open_file = open(file_name, "wb")
  pickle.dump(py_obj, open_file)
  open_file.close()

In [5]:
def readPickleEncodingsFromFile(file_name:str):
  """
  This function read a Pickle encoded file and returns the decoded python object.
  INPUTS:
    file_name:
      name of the file from which encodings have to read.
  OUPUTS:
    returns a python object with decoded pickle file
  """
  open_file = open(file_name, "rb")
  decode_obj = pickle.load(open_file)
  open_file.close()
  return decode_obj

In [6]:
def faceEncodings(images):
  """
  this function takes a list of images and return list of encoded images.
  INPUTS:
    images: list of images (numpy array)
  OutPUTS:
    list of encoded images
  """
  encodeList = []
  for img in images:
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    encode = face_recognition.face_encodings(img)[0]
    encodeList.append(encode)
  return encodeList

In [10]:
def EncodeFaces(path_to_std_img_dir, std_names_file_name, std_enc_file_name):
  """
  This function encode all the images in a folder and save the encodings in pickel form to the specified folder.
  INPUTS:
    path_to_std_img_dir: 
      reletive or absolute path to folder in which all the student images are present
    
    std_names_file_name: 
      file name to store all the student names in pickle encoded form 
    
    std_enc_file_name:
      file name to store encodings of students faces in pickle form
  OUPUTS:
    it returns nothing, but save the results to files in pickle form
  """
  
  images = []
  personNames = []
  myList = os.listdir(path_to_std_img_dir)
  for cu_img in myList:
    current_Img = cv2.imread(f'{path_to_std_img_dir}/{cu_img}')
    images.append(current_Img)
    personNames.append(os.path.splitext(cu_img)[0])
  print(personNames)

  savePickleEncodingsToFile(std_names_file_name, personNames)
  # open_file = open(std_names_file_name, "wb")
  # pickle.dump(personNames, open_file)
  # open_file.close()

  encodeListKnown = faceEncodings(images)

  savePickleEncodingsToFile(std_enc_file_name, encodeListKnown)
  # open_file = open(std_enc_file_name, "wb")
  # pickle.dump(encodeListKnown, open_file)
  # open_file.close()


In [11]:
StudentFaces_dir = "/content/drive/MyDrive/ML_Project/Student_Faces"
Name_list = "/content/drive/MyDrive/ML_Project/StudentName.pkl"
StudentFaces_Encoding = "/content/drive/MyDrive/ML_Project/StudentEncoding.pkl"
ClassroomImagePath = "/content/drive/MyDrive/ML_Project/ClassroomPicture/IMG_20220315_095343_1.jpg"


In [13]:
EncodeFaces(StudentFaces_dir, Name_list, StudentFaces_Encoding)

['Abdullah', 'Usama', 'abdurrehman', 'Asfand', 'Shaheer', 'AbdullahTaqueer', 'Abdullah1', 'Abdurrehman2', 'AliYaqteen', 'BilalShah', 'Raza', 'shaheer2', 'Ahmreen', 'Maheen', 'Armaghan', 'Asfandyar', 'Bilalhassan', 'AhmadZahid', 'Iritiza', 'AleenaAmjad', 'Armaghan1']


In [16]:
def FindFace(enc_names_file, enc_std_faces_file, class_room_img):
  """
  This function detects and recongnize faces in the image of whole class 
  INPUTS:
  enc_names_file:
    pickle encoded file name which have names of student in it.
  enc_std_faces_file:
    pickle encoded file name which have all the face encodings of students in it.
  class_room_img:
    a camera taken image file of the whole class for attendence.
  OUTPUTS:
    it saves the class room images with each face bounding box and there name, if name is in black it is much accurate
    if name is in red it is not much sure about them
  """

  Filename = os.path.basename(class_room_img)
  # file_name = "/content/drive/MyDrive/ML_Project/StudentName.pkl"

  personNames = readPickleEncodingsFromFile(enc_names_file)
  # open_file = open(enc_names_file, "rb")
  # personNames = pickle.load(open_file)
  # open_file.close()


  # file_name = "/content/drive/MyDrive/ML_Project/StudentEncoding.pkl"
  encodeListKnown = readPickleEncodingsFromFile(enc_std_faces_file)
  # open_file = open(enc_std_faces_file, "rb")
  # encodeListKnown = pickle.load(open_file)
  # open_file.close()

  # Path = '/content/drive/MyDrive/ML_Project/ClassroomPicture/IMG_20220315_095343_1.jpg'


  image = face_recognition.load_image_file(class_room_img)


  detector = MTCNN()

  faces = detector.detect_faces(image)

  face_location1 = []
  for face in faces:
    x,y,w,h = face['box']

    x1 = y
    w1 = y+h
    h1 = x
    y1 = x+w

    a = (x1, y1, w1, h1)
    face_location1.append(a)


  encodesCurrentFrame = face_recognition.face_encodings(image, face_location1)

  print('Total Faces detected: ', len(face_location1))

  ImageContours = cv2.imread(class_room_img)
  ImageContours = cv2.cvtColor(ImageContours,cv2.COLOR_BGR2RGB)

  ReplicateImage = ImageContours.copy()

  for cntr in face_location1:
      x,y,w,h = cntr
      cv2.rectangle(ReplicateImage, (h, x), (y, w), (0, 0, 255), 2)

  cv2.imwrite('/content/drive/MyDrive/ML_Project/AttendanceRecord/'+ Filename ,ReplicateImage)
  # cv2.imwrite('/content/drive/MyDrive/ML_Project/Output_Image/Marked_Face_Image.jpg',ReplicateImage)

  Name_Dist = {}
  Not_sure = []
  # Not_sure1 = []
  for encodeFace, faceLoc in zip(encodesCurrentFrame, face_location1):
          matches = face_recognition.compare_faces(encodeListKnown, encodeFace)
          
          faceDis = face_recognition.face_distance(encodeListKnown, encodeFace)
          # print(faceDis)
          matchIndex = np.argmin(faceDis)
          match_threshold = np.min(faceDis)
          # print(matchIndex)
          if faceDis[matchIndex] < 0.50:
              name = personNames[matchIndex].upper()
              # print(name)
              Name_Dist[name] = [match_threshold,faceLoc]
          else:
              name = personNames[matchIndex].upper()
              
              if name not in Not_sure:
                if name not in Name_Dist:

                  enc = (name, match_threshold,faceLoc)
                  Not_sure.append(enc)
              


              # print('Unknown')
              # Name_Dist['Unknown'] = faceLoc
  NOT_SURE_DIST = {}

  for Face in Not_sure:
    
    Name = Face[0]
    if Name not in Name_Dist:
      prop = []
      prop.append(Face[1])
      for temp_Face in Not_sure:
        if Name == temp_Face[0]:
          prop.append(Face[1])
      min = np.min(prop)

      for temp in Not_sure:
        if temp[0] == Name and temp[1] == min:
          NOT_SURE_DIST[Name] = [temp[1] ,temp[2]]
          # print(Name)




  image = Image.open('/content/drive/MyDrive/ML_Project/AttendanceRecord/'+ Filename)

  draw = ImageDraw.Draw(image)

  font = ImageFont.truetype('/content/drive/MyDrive/ML_Project/Roboto[wdth,wght].ttf', size=45)

  for key in Name_Dist:
    A = Name_Dist[key][1]
    print(A)
    (x, y) = (A[1], A[0])
    message = str(key)
    color = 'rgb(0, 0, 0)'
    # print(x,y)
    draw.text((x, y), message, fill=color, font=font)

  for key in NOT_SURE_DIST:
    A = NOT_SURE_DIST[key][1]
    (x, y) = (A[1], A[0])
    message = str(key)
    color = 'rgb(255, 0, 0)'
    # print(x,y)
    draw.text((x, y), message, fill=color, font=font)
  
  image.save('/content/drive/MyDrive/ML_Project/AttendanceRecord/'+ Filename)


In [17]:
FindFace(Name_list, StudentFaces_Encoding, ClassroomImagePath)

Total Faces detected:  21
(1319, 1819, 1456, 1710)
(699, 2258, 758, 2209)
(927, 2356, 1014, 2282)
(483, 2142, 517, 2115)
(789, 1183, 847, 1132)
(690, 1881, 741, 1841)
(915, 2007, 990, 1946)
(607, 1525, 648, 1493)
(685, 1255, 725, 1221)
(609, 1286, 638, 1259)
